In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv('../data/Modelar_UH2022.txt', sep = '|')

In [3]:
df.head(10)

,ID,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH
0,0,2019-06-13 08:34:09,369320,0.0,17,0.0
1,0,2019-06-13 17:34:10,369403,0.0,2,0.0
2,0,2019-06-13 18:34:10,369403,0.0,0,0.0
3,0,2019-06-13 04:34:10,369284,0.0,1,0.0
4,0,2019-06-13 14:34:10,369356,0.0,28,0.0
5,0,2019-06-13 01:34:09,369283,0.0,0,0.0
6,0,2019-06-13 16:34:10,369401,0.0,42,0.0
7,0,2019-06-13 10:34:09,369323,0.0,0,0.0
8,0,2019-06-13 22:34:08,369511,0.0,15,0.0
9,0,2019-06-13 23:34:08,369552,0.0,41,0.0


In [5]:
df.isnull().sum()

ID                        0
SAMPLETIME                0
READINGINTEGER            0
READINGTHOUSANDTH    140056
DELTAINTEGER              0
DELTATHOUSANDTH      140056
dtype: int64

In [4]:
df = df.drop(labels = ['READINGTHOUSANDTH', 'DELTATHOUSANDTH', 'READINGINTEGER'], axis=1)

In [5]:
df['SAMPLETIME'] = pd.to_datetime(df['SAMPLETIME'])

Cuando la hora no está informada, la imputa a 00:00:00.

In [6]:
# Add year
df['year'] = df['SAMPLETIME'].dt.year

In [7]:
# Add month and month name
df['month'] = df['SAMPLETIME'].dt.month

In [8]:
# Add day, day of week and day name
df['day'] = df['SAMPLETIME'].dt.day
df['dayofweek'] = df['SAMPLETIME'].dt.dayofweek

In [9]:
# Is weekend?
df['date_is_weekday'] = np.where(df['dayofweek'].isin([6, 7]), -1, 1)
#En vez de hacer variable binaria, se pone entre -1 y 1, para que esté "alineada", con las normalizaciones por senos y cosenos, para el futuro cluster.

In [10]:
# Week of the year
df['week'] = df['SAMPLETIME'].dt.week

C:\Users\Manuel\AppData\Local\Temp/ipykernel_5504/3935838887.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] = df['SAMPLETIME'].dt.week


In [19]:
#Day from 1 to 365
df['day_1-365'] = df['SAMPLETIME'].dt.dayofyear

In [20]:
df

,ID,SAMPLETIME,DELTAINTEGER,year,month,day,dayofweek,date_is_weekday,week,day_1-365
0,0,2019-06-13 08:34:09,17,2019,6,13,3,1,24,164
1,0,2019-06-13 17:34:10,2,2019,6,13,3,1,24,164
2,0,2019-06-13 18:34:10,0,2019,6,13,3,1,24,164
3,0,2019-06-13 04:34:10,1,2019,6,13,3,1,24,164
4,0,2019-06-13 14:34:10,28,2019,6,13,3,1,24,164
...,...,...,...,...,...,...,...,...,...,...
21404823,2749,2019-10-11 06:27:14,0,2019,10,11,4,1,41,284
21404824,2756,2019-04-06 05:50:17,0,2019,4,6,5,1,14,96
21404825,2756,2019-04-06 07:50:17,12,2019,4,6,5,1,14,96
21404826,2756,2019-04-06 04:50:17,0,2019,4,6,5,1,14,96


Utilizar seno y coseno para conservar distancias temporales.

En algunos casos tiene sentido mantener la distancia temporal.

Por ejemplo, entre los meses del año. La distancia. Sin embargo, si utilizamos su representación numérica tradicional estaremos informando a nuestro sistema que enero y diciembre son los meses más distanciados.

Para ello vamos a normalizar la variable month y aplicarle la función coseno:

In [77]:
df.dtypes
df['month'].value_counts()
df['month_norm'] = 2 * math.pi * df['month']/max(df['month'])
df['month_cos'] = np.cos(df["month_norm"])
df['month_sin'] = np.sin(df["month_norm"])

In [78]:
df[['month', 'month_norm', 'month_cos', 'month_sin']]

,month,month_norm,month_cos,month_sin
0,6,3.141593,-1.0,1.224647e-16
1,6,3.141593,-1.0,1.224647e-16
2,6,3.141593,-1.0,1.224647e-16
3,6,3.141593,-1.0,1.224647e-16
4,6,3.141593,-1.0,1.224647e-16
...,...,...,...,...
21404823,10,5.235988,0.5,-8.660254e-01
21404824,4,2.094395,-0.5,8.660254e-01
21404825,4,2.094395,-0.5,8.660254e-01
21404826,4,2.094395,-0.5,8.660254e-01


In [21]:
df.sort_values(by=['SAMPLETIME'])

,ID,SAMPLETIME,DELTAINTEGER,year,month,day,dayofweek,date_is_weekday,week,day_1-365
8362807,978,2019-02-01 00:00:00,0,2019,2,1,4,1,5,32
18229024,2144,2019-02-01 00:00:00,0,2019,2,1,4,1,5,32
15250776,1791,2019-02-01 00:00:00,0,2019,2,1,4,1,5,32
5241329,610,2019-02-01 00:00:00,5,2019,2,1,4,1,5,32
19230973,2263,2019-02-01 00:00:00,3,2019,2,1,4,1,5,32
...,...,...,...,...,...,...,...,...,...,...
21310855,2668,2020-01-31 23:59:59,0,2020,1,31,4,1,5,31
21243482,2639,2020-01-31 23:59:59,0,2020,1,31,4,1,5,31
21290093,2660,2020-01-31 23:59:59,11,2020,1,31,4,1,5,31
21019857,2548,2020-01-31 23:59:59,24,2020,1,31,4,1,5,31


In [79]:
df['day'].value_counts()
df['day_norm'] = 2 * math.pi * df['day']/max(df['day'])
df['day_cos'] = np.cos(df["day_norm"])
df['day_sin'] = np.sin(df["day_norm"])

In [80]:
df[['day', 'day_norm', 'day_cos', 'day_sin']]

,day,day_norm,day_cos,day_sin
0,13,2.634884,-0.874347,0.485302
1,13,2.634884,-0.874347,0.485302
2,13,2.634884,-0.874347,0.485302
3,13,2.634884,-0.874347,0.485302
4,13,2.634884,-0.874347,0.485302
...,...,...,...,...
21404823,11,2.229517,-0.612106,0.790776
21404824,6,1.216100,0.347305,0.937752
21404825,6,1.216100,0.347305,0.937752
21404826,6,1.216100,0.347305,0.937752


In [81]:
df['delta_norm_0-1'] = (df['day'] - df['day'].min()) / (df['day'].max() - df['day'].min())

In [82]:
df

,ID,SAMPLETIME,DELTAINTEGER,year,month,day,dayofweek,date_is_weekday,week,month_norm,month_cos,month_sin,day_norm,day_cos,day_sin,delta_norm_0-1
0,0,2019-06-13 08:34:09,17,2019,6,13,3,1,24,3.141593,-1.0,1.224647e-16,2.634884,-0.874347,0.485302,0.400000
1,0,2019-06-13 17:34:10,2,2019,6,13,3,1,24,3.141593,-1.0,1.224647e-16,2.634884,-0.874347,0.485302,0.400000
2,0,2019-06-13 18:34:10,0,2019,6,13,3,1,24,3.141593,-1.0,1.224647e-16,2.634884,-0.874347,0.485302,0.400000
3,0,2019-06-13 04:34:10,1,2019,6,13,3,1,24,3.141593,-1.0,1.224647e-16,2.634884,-0.874347,0.485302,0.400000
4,0,2019-06-13 14:34:10,28,2019,6,13,3,1,24,3.141593,-1.0,1.224647e-16,2.634884,-0.874347,0.485302,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21404823,2749,2019-10-11 06:27:14,0,2019,10,11,4,1,41,5.235988,0.5,-8.660254e-01,2.229517,-0.612106,0.790776,0.333333
21404824,2756,2019-04-06 05:50:17,0,2019,4,6,5,1,14,2.094395,-0.5,8.660254e-01,1.216100,0.347305,0.937752,0.166667
21404825,2756,2019-04-06 07:50:17,12,2019,4,6,5,1,14,2.094395,-0.5,8.660254e-01,1.216100,0.347305,0.937752,0.166667
21404826,2756,2019-04-06 04:50:17,0,2019,4,6,5,1,14,2.094395,-0.5,8.660254e-01,1.216100,0.347305,0.937752,0.166667


In [83]:
is_2019 =  df['year'] == 2019
df_2019 =  df[is_2019]

In [84]:
df_2019['year'].value_counts()

2019    19454777
Name: year, dtype: int64

In [85]:
df_cluster = df.drop(labels = ['DELTAINTEGER', 'SAMPLETIME', 'year', 'month', 'day', 'dayofweek', 'week', 'month_norm', 'day_norm'], axis=1)

In [86]:
df_cluster.head(10)

,ID,date_is_weekday,month_cos,month_sin,day_cos,day_sin,delta_norm_0-1
0,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
1,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
2,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
3,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
4,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
5,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
6,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
7,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
8,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4
9,0,1,-1.0,1.224647e-16,-0.874347,0.485302,0.4


In [88]:
df_cluster.to_csv('../data/df_para_cluster.csv')